#LLM_MODEL

##LIBS

In [4]:
from dotenv import load_dotenv
load_dotenv()
import re
from botocore.exceptions import ClientError
import json
from botocore.exceptions import ClientError
import boto3
import os
import json
import logging
import boto3
import time
import json
import uuid
import pprint
import os
import time
import yfinance as yf
import pandas as pd
import matplotlib.pyplot as plt
from io import BytesIO
from PIL import Image
import yfinance as yf
import pandas as pd
import matplotlib.pyplot as plt
import fear_and_greed
import json
from io import BytesIO
from PIL import Image
import base64
from opensearchpy import OpenSearch, RequestsHttpConnection
from requests_aws4auth import AWS4Auth

#from utils import hints



In [5]:
DEFAULT_REGION = "us-west-2"
ACCESS_KEY = os.getenv("AWS_ACCESS_KEY")
SECRET_KEY = os.getenv("AWS_SECRET_KEY")
SESSION_TOKEN = os.getenv("AWS_SESSION_TOKEN")

In [6]:
region = os.getenv('AWS_DEFAULT_REGION')

bedrock_runtime = boto3.client(
    'bedrock-runtime',
    region_name=region,
    aws_access_key_id=ACCESS_KEY,
    aws_secret_access_key=SECRET_KEY,
     aws_session_token=SESSION_TOKEN
) 

##Boto3_setting

In [7]:
# setting logger
logging.basicConfig(format='[%(asctime)s] p%(process)s {%(filename)s:%(lineno)d} %(levelname)s - %(message)s', level=logging.INFO)
logger = logging.getLogger(__name__)

In [8]:
# getting boto3 clients for required AWS services
sts_client = boto3.client('sts')
iam_client = boto3.client('iam')
s3_client = boto3.client('s3')
lambda_client = boto3.client('lambda')
bedrock_agent_client = boto3.client('bedrock-agent')
bedrock_agent_runtime_client = boto3.client('bedrock-agent-runtime')
open_search_serverless_client = boto3.client('opensearchserverless')

In [9]:
session = boto3.session.Session()
region = session.region_name
account_id = sts_client.get_caller_identity()["Account"]
region, account_id

('us-west-2', '939490215743')

In [10]:
suffix = int(time.time())


In [11]:
# Generate random prefix for unique IAM roles, agent name and S3 Bucket and 
# assign variables
suffix = f"{region}-{account_id}"

suffix = int(time.time())
agent_name = f"bedrock-docs-kb-agents{suffix}"

agent_alias_name = f"bedrock-docs-alias{suffix}"




#bucket_name = f'{agent_name}-{suffix}'

bucket_name = f'my-data-bucket-2024'
bucket_arn = f"arn:aws:s3:::{bucket_name}"



#bedrock_agent_bedrock_allow_policy_name = f"bda-bedrock-allow-{suffix}"

#bedrock_agent_s3_allow_policy_name = f"bda-s3-allow-{suffix}"

#bedrock_agent_kb_allow_policy_name = f"bda-kb-allow-{suffix}"




  # Utilise l'horodatage actuel comme suffixe

bedrock_agent_bedrock_allow_policy_name = f"bda-bedrock-allow-{suffix}"
bedrock_agent_s3_allow_policy_name = f"bda-s3-allow-{suffix}"
bedrock_agent_kb_allow_policy_name = f"bda-kb-allow-{suffix}"



agent_role_name = f'AmazonBedrockExecutionRoleForAgents_bedrock_docs{suffix}'

#kb_name = f'bedrock-docs-kb-{suffix}'
# Cela crée un nombre unique basé sur l'heure actuelle

kb_name = f'bedrock-docs-kb-{suffix}'

data_source_name = f'bedrock-docs-kb-docs-{suffix}'
kb_files_path = 'kb_documents'
kb_key = 'kb_documents'

kb_role_name = f'rapport1_test_docs{suffix}'



kb_bedrock_allow_policy_name = f"bd-kb-bedrock-allow-{suffix}"

kb_aoss_allow_policy_name = f"bd-kb-aoss-allow-{suffix}"

kb_s3_allow_policy_name = f"bd-kb-s3-allow-{suffix}"

kb_collection_name = f'bd-kbc-{suffix}'


##knowledge_base_Setup


In [12]:

# Select Amazon titan as the embedding model
embedding_model_arn = f'arn:aws:bedrock:{region}::foundation-model/amazon.titan-embed-text-v1'
kb_vector_index_name = "bedrock-knowledge-base-index"
kb_metadataField = 'bedrock-knowledge-base-metadata'
kb_textField = 'bedrock-knowledge-base-text'
kb_vectorField = 'bedrock-knowledge-base-vector'
model_id = "anthropic.claude-3-sonnet-20240229-v1:0"

# agent configuration
agent_instruction ="you ara a financial chatbot helping in analysing data , answer briefly and refer well to the data given to you"
#"""
#Analyze the provided financial report to generate a concise summary that includes key information such as revenue, expenses, profit margins, and significant changes compared to previous periods. Additionally, evaluate the sentiment in the report, focusing on the letters to shareholders and management commentary. Determine whether the sentiment is optimistic, pessimistic, or neutral, and present your findings in a structured format. Include relevant quotes or phrases to support your sentiment analysis and provide visualizations if applicable. la reponse final doit etre dans un format json , avec 3 cles 
#"""

In [13]:
print(bucket_name)

my-data-bucket-2024


In [14]:
try:
    if region != 'us-east-1':
        s3_client.create_bucket(
            Bucket=bucket_name.lower(),
            CreateBucketConfiguration={'LocationConstraint': region}
        )
    else:
        s3_client.create_bucket(Bucket=bucket_name)
except Exception as e:
    if "Bucket" in str(e):
        pass
    else:
        raise e

##UPLOAD_RAG_DATA for Fundamental and general analysis

In [15]:


#kb_documents/Rapport_Act_2018_Fr.pdf
#kb_documents
# Upload PDF files to S3
kb_files_path = 'financedata'
for f in os.listdir(kb_files_path):
    if f.endswith(".pdf"):
        local_file_path = os.path.join(kb_files_path, f)
        s3_client.upload_file(local_file_path, bucket_name, os.path.join(kb_key, f))
        print(f"Uploaded {f} to s3://{bucket_name}/{kb_key}/{f}")


Uploaded 2023-rapport-financier-annuel-bce.pdf to s3://my-data-bucket-2024/kb_documents/2023-rapport-financier-annuel-bce.pdf
Uploaded Rogers-2023-Annual-Report.pdf to s3://my-data-bucket-2024/kb_documents/Rogers-2023-Annual-Report.pdf
Uploaded 2020-rapport-annuel-bce.pdf to s3://my-data-bucket-2024/kb_documents/2020-rapport-annuel-bce.pdf
Uploaded Rogers-2022-Annual-Report-1.pdf to s3://my-data-bucket-2024/kb_documents/Rogers-2022-Annual-Report-1.pdf
Uploaded 2022-annuel-10Q-FR-09122022-Final.pdf to s3://my-data-bucket-2024/kb_documents/2022-annuel-10Q-FR-09122022-Final.pdf
Uploaded Hydro One Limited Annual Report 2022.pdf to s3://my-data-bucket-2024/kb_documents/Hydro One Limited Annual Report 2022.pdf
Uploaded 9465D_Rapport_Annuel_2023_CCA_FR_F.pdf to s3://my-data-bucket-2024/kb_documents/9465D_Rapport_Annuel_2023_CCA_FR_F.pdf
Uploaded 2022-rapport-financier-annuel-bce.pdf to s3://my-data-bucket-2024/kb_documents/2022-rapport-financier-annuel-bce.pdf
Uploaded Hydro One Limited Annua

In [16]:
# Upload Knowledge Base files to thikb_files_paths s3 bucket

#for f in os.listdir(kb_files_path):
    #if f.endswith(".pdf"):
        #s3_client.upload_file(kb_files_path+'/'+f, bucket_name, kb_key+'/'+f)

##POLICIES

In [17]:
import boto3

iam_client = boto3.client('iam')
policies = iam_client.list_policies(Scope='Local')  # List only your policies

for policy in policies['Policies']:
    print(policy['PolicyName'])


bd-kb-s3-allow-1730624761
bd-kb-bedrock-allow-1730646173
bd-kb-aoss-allow-1730670623
bda-bedrock-allow-1730670623
bd-kb-bedrock-allow-1730624479
bda-bedrock-allow-1730646923
bd-kb-bedrock-allow-1730609702
bda-bedrock-allow-1730624479
iam_policy-0
bda-bedrock-allow-1730609928
bda-bedrock-allow-1730624761
bda-kb-allow-1730624761
bda-kb-allow-1730624479
bda-bedrock-allow-1730612149
AmazonSageMaker-ExecutionPolicy-20241102T134231
bda-kb-allow-1730731847
bda-bedrock-allow-1730731847
bd-kb-bedrock-allow-1730615197
bd-kb-s3-allow-1730615197
AmazonSageMakerServiceCatalogProductsUseRole-20241102T134284
bd-kb-bedrock-allow-1730664838
bd-kb-bedrock-allow-1730612149
bd-kb-aoss-allow-1730623728
bda-bedrock-allow-us-west-2-939490215743
bd-kb-s3-allow-1730731847
bda-kb-allow-1730610699
AmazonBedrockFoundationModelPolicyForKnowledgeBase_9394
bd-kb-bedrock-allow-1730670623
bd-kb-bedrock-allow-us-west-2-939490215743
bd-kb-aoss-allow-1730613481
bd-kb-aoss-allow-1730609702
bd-kb-s3-allow-1730664838
bd-kb-

In [18]:
# Create IAM policies for KB to invoke embedding model
bedrock_kb_allow_fm_model_policy_statement = {
    "Version": "2012-10-17",
    "Statement": [
        {
            "Sid": "AmazonBedrockAgentBedrockFoundationModelPolicy",
            "Effect": "Allow",
            "Action": "bedrock:InvokeModel",
            "Resource": [
                embedding_model_arn
            ]
        }
    ]
}

kb_bedrock_policy_json = json.dumps(bedrock_kb_allow_fm_model_policy_statement)

kb_bedrock_policy = iam_client.create_policy(
    PolicyName=kb_bedrock_allow_policy_name,
    PolicyDocument=kb_bedrock_policy_json
)

In [19]:
# Create IAM policies for KB to access OpenSearch Serverless
bedrock_kb_allow_aoss_policy_statement = {
    "Version": "2012-10-17",
    "Statement": [
        {
            "Effect": "Allow",
            "Action": "aoss:APIAccessAll",
            "Resource": [
                f"arn:aws:aoss:{region}:{account_id}:collection/*"
            ]
        }
    ]
}


kb_aoss_policy_json = json.dumps(bedrock_kb_allow_aoss_policy_statement)

kb_aoss_policy = iam_client.create_policy(
    PolicyName=kb_aoss_allow_policy_name,
    PolicyDocument=kb_aoss_policy_json
)

In [20]:
kb_s3_allow_policy_statement = {
    "Version": "2012-10-17",
    "Statement": [
        {
            "Sid": "AllowKBAccessDocuments",
            "Effect": "Allow",
            "Action": [
                "s3:GetObject",
                "s3:ListBucket"
            ],
            "Resource": [
                f"arn:aws:s3:::{bucket_name}/*",
                f"arn:aws:s3:::{bucket_name}"
            ],
            "Condition": {
                "StringEquals": {
                    "aws:ResourceAccount": f"{account_id}"
                }
            }
        }
    ]
}


kb_s3_json = json.dumps(kb_s3_allow_policy_statement)
kb_s3_policy = iam_client.create_policy(
    PolicyName=kb_s3_allow_policy_name,
    PolicyDocument=kb_s3_json
)

In [21]:
# Create IAM Role for the agent and attach IAM policies
assume_role_policy_document = {
    "Version": "2012-10-17",
    "Statement": [{
          "Effect": "Allow",
          "Principal": {
            "Service": "bedrock.amazonaws.com"
          },
          "Action": "sts:AssumeRole"
    }]
}



In [22]:
assume_role_policy_document_json = json.dumps(assume_role_policy_document)


In [23]:
kb_role = iam_client.create_role(
    RoleName=kb_role_name,
    AssumeRolePolicyDocument=assume_role_policy_document_json
)



In [24]:
# Pause to make sure role is created
time.sleep(10)
    
iam_client.attach_role_policy(
    RoleName=kb_role_name,
    PolicyArn=kb_bedrock_policy['Policy']['Arn']
)

iam_client.attach_role_policy(
    RoleName=kb_role_name,
    PolicyArn=kb_aoss_policy['Policy']['Arn']
)

iam_client.attach_role_policy(
    RoleName=kb_role_name,
    PolicyArn=kb_s3_policy['Policy']['Arn']
)

{'ResponseMetadata': {'RequestId': 'd016bad7-6d89-4722-9990-f663f750a26e',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Mon, 04 Nov 2024 16:13:59 GMT',
   'x-amzn-requestid': 'd016bad7-6d89-4722-9990-f663f750a26e',
   'content-type': 'text/xml',
   'content-length': '212'},
  'RetryAttempts': 0}}

In [25]:
import boto3

iam_client = boto3.client('iam')

roles = iam_client.list_roles()
for role in roles['Roles']:
    print(role['RoleName'])


AmazonBedrockExecutionRoleForAgents_bedrock_docs1730623728
AmazonBedrockExecutionRoleForAgents_bedrock_docs1730624479
AmazonBedrockExecutionRoleForAgents_bedrock_docs1730624761
AmazonBedrockExecutionRoleForAgents_bedrock_docs1730646173
AmazonBedrockExecutionRoleForAgents_bedrock_docs1730646923
AmazonBedrockExecutionRoleForAgents_bedrock_docs1730664838
AmazonBedrockExecutionRoleForAgents_bedrock_docs1730670623
AmazonBedrockExecutionRoleForAgents_bedrock_docs1730731847
AmazonBedrockExecutionRoleForAgents_booking-agent
AmazonBedrockExecutionRoleForKnowledgeBase_9394
AmazonSageMakerServiceCatalogProductsLaunchRole
AmazonSageMakerServiceCatalogProductsUseRole
AWSServiceRoleForAmazonEMRServerless
AWSServiceRoleForAmazonOpenSearchServerless
AWSServiceRoleForAmazonOpenSearchService
AWSServiceRoleForAmazonSageMakerNotebooks
AWSServiceRoleForAWSCloud9
AWSServiceRoleForAWSLicenseManagerMemberAccountRole
AWSServiceRoleForAWSLicenseManagerRole
AWSServiceRoleForAWSServiceCatalogAppRegistry
AWSServic

In [26]:
kb_role_arn = kb_role["Role"]["Arn"]
kb_role_arn

'arn:aws:iam::939490215743:role/rapport1_test_docs1730736817'

##Create_collection

In [27]:
# Create OpenSearch Collection
security_policy_json = {
    "Rules": [
        {
            "ResourceType": "collection",
            "Resource":[
                f"collection/{kb_collection_name}"
            ]
        }
    ],
    "AWSOwnedKey": True
}
security_policy = open_search_serverless_client.create_security_policy(
    description='security policy of aoss collection',
    name=kb_collection_name,
    policy=json.dumps(security_policy_json),
    type='encryption'
)

In [28]:
network_policy_json = [
  {
    "Rules": [
      {
        "Resource": [
          f"collection/{kb_collection_name}"
        ],
        "ResourceType": "dashboard"
      },
      {
        "Resource": [
          f"collection/{kb_collection_name}"
        ],
        "ResourceType": "collection"
      }
    ],
    "AllowFromPublic": True
  }
]

network_policy = open_search_serverless_client.create_security_policy(
    description='network policy of aoss collection',
    name=kb_collection_name,
    policy=json.dumps(network_policy_json),
    type='network'
)

In [29]:
response = sts_client.get_caller_identity()
current_role = response['Arn']
current_role

'arn:aws:sts::939490215743:assumed-role/WSParticipantRole/Participant'

In [30]:
data_policy_json = [
  {
    "Rules": [
      {
        "Resource": [
          f"collection/{kb_collection_name}"
        ],
        "Permission": [
          "aoss:DescribeCollectionItems",
          "aoss:CreateCollectionItems",
          "aoss:UpdateCollectionItems",
          "aoss:DeleteCollectionItems"
        ],
        "ResourceType": "collection"
      },
      {
        "Resource": [
          f"index/{kb_collection_name}/*"
        ],
        "Permission": [
            "aoss:CreateIndex",
            "aoss:DeleteIndex",
            "aoss:UpdateIndex",
            "aoss:DescribeIndex",
            "aoss:ReadDocument",
            "aoss:WriteDocument"
        ],
        "ResourceType": "index"
      }
    ],
    "Principal": [
        kb_role_arn,
        f"arn:aws:sts::{account_id}:assumed-role/Admin/*",
        current_role
    ],
    "Description": ""
  }
]

data_policy = open_search_serverless_client.create_access_policy(
    description='data access policy for aoss collection',
    name=kb_collection_name,
    policy=json.dumps(data_policy_json),
    type='data'
)

In [31]:
opensearch_collection_response = open_search_serverless_client.create_collection(
    description='OpenSearch collection for Amazon Bedrock Knowledge Base',
    name=kb_collection_name,
    standbyReplicas='DISABLED',
    type='VECTORSEARCH'
)
opensearch_collection_response

{'createCollectionDetail': {'arn': 'arn:aws:aoss:us-west-2:939490215743:collection/nzik8pithijvalasgq19',
  'createdDate': 1730736840880,
  'description': 'OpenSearch collection for Amazon Bedrock Knowledge Base',
  'id': 'nzik8pithijvalasgq19',
  'kmsKeyArn': 'auto',
  'lastModifiedDate': 1730736840880,
  'name': 'bd-kbc-1730736817',
  'standbyReplicas': 'DISABLED',
  'status': 'CREATING',
  'type': 'VECTORSEARCH'},
 'ResponseMetadata': {'RequestId': '33e7f18b-65ab-429e-8fe8-90b80cefecc0',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '33e7f18b-65ab-429e-8fe8-90b80cefecc0',
   'date': 'Mon, 04 Nov 2024 16:14:00 GMT',
   'content-type': 'application/x-amz-json-1.0',
   'content-length': '382',
   'connection': 'keep-alive'},
  'RetryAttempts': 0}}

In [32]:
collection_arn = opensearch_collection_response["createCollectionDetail"]["arn"]
collection_arn

'arn:aws:aoss:us-west-2:939490215743:collection/nzik8pithijvalasgq19'

In [33]:
# wait for collection creation
response = open_search_serverless_client.batch_get_collection(names=[kb_collection_name])
# Periodically check collection status
while (response['collectionDetails'][0]['status']) == 'CREATING':
    print('Creating collection...')
    time.sleep(30)
    response = open_search_serverless_client.batch_get_collection(names=[kb_collection_name])
print('\nCollection successfully created:')
print(response["collectionDetails"])
# Extract the collection endpoint from the response
host = (response['collectionDetails'][0]['collectionEndpoint'])
final_host = host.replace("https://", "")
final_host

Creating collection...

Collection successfully created:
[{'arn': 'arn:aws:aoss:us-west-2:939490215743:collection/nzik8pithijvalasgq19', 'collectionEndpoint': 'https://nzik8pithijvalasgq19.us-west-2.aoss.amazonaws.com', 'createdDate': 1730736840880, 'dashboardEndpoint': 'https://nzik8pithijvalasgq19.us-west-2.aoss.amazonaws.com/_dashboards', 'description': 'OpenSearch collection for Amazon Bedrock Knowledge Base', 'id': 'nzik8pithijvalasgq19', 'kmsKeyArn': 'auto', 'lastModifiedDate': 1730736864158, 'name': 'bd-kbc-1730736817', 'standbyReplicas': 'DISABLED', 'status': 'ACTIVE', 'type': 'VECTORSEARCH'}]


'nzik8pithijvalasgq19.us-west-2.aoss.amazonaws.com'

In [34]:
credentials = boto3.Session().get_credentials()
service = 'aoss'
awsauth = AWS4Auth(
    credentials.access_key, 
    credentials.secret_key,
    region, 
    service, 
    session_token=credentials.token
)

# Build the OpenSearch client
open_search_client = OpenSearch(
    hosts=[{'host': final_host, 'port': 443}],
    http_auth=awsauth,
    use_ssl=True,
    verify_certs=True,
    connection_class=RequestsHttpConnection,
    timeout=300
)
# It can take up to a minute for data access rules to be enforced
time.sleep(45)
index_body = {
    "settings": {
        "index.knn": True,
        "number_of_shards": 1,
        "knn.algo_param.ef_search": 512,
        "number_of_replicas": 0,
    },
    "mappings": {
        "properties": {}
    }
}

index_body["mappings"]["properties"][kb_vectorField] = {
    "type": "knn_vector",
    "dimension": 1536,
    "method": {
         "name": "hnsw",
         "engine": "faiss"
    },
}

index_body["mappings"]["properties"][kb_textField] = {
    "type": "text"
}

index_body["mappings"]["properties"][kb_metadataField] = {
    "type": "text"
}

# Create index
response = open_search_client.indices.create(kb_vector_index_name, body=index_body)
print('\nCreating index:')
print(response)

[2024-11-04 11:14:31,260] p197733 {credentials.py:1147} INFO - Found credentials in environment variables.
[2024-11-04 11:15:22,108] p197733 {base.py:258} INFO - PUT https://nzik8pithijvalasgq19.us-west-2.aoss.amazonaws.com:443/bedrock-knowledge-base-index [status:200 request:5.847s]



Creating index:
{'acknowledged': True, 'shards_acknowledged': True, 'index': 'bedrock-knowledge-base-index'}


##Knowledgre_base_storage

In [35]:
storage_configuration = {
    'opensearchServerlessConfiguration': {
        'collectionArn': collection_arn, 
        'fieldMapping': {
            'metadataField': kb_metadataField,
            'textField': kb_textField,
            'vectorField': kb_vectorField
        },
        'vectorIndexName': kb_vector_index_name
    },
    'type': 'OPENSEARCH_SERVERLESS'
}

In [36]:
# Creating the knowledge base
try:
    # ensure the index is created and available
    time.sleep(45)
    kb_obj = bedrock_agent_client.create_knowledge_base(
        name=kb_name, 
        description='KB that contains the bedrock documentation',
        roleArn=kb_role_arn,
        knowledgeBaseConfiguration={
            'type': 'VECTOR',  # Corrected type
            'vectorKnowledgeBaseConfiguration': {
                'embeddingModelArn': embedding_model_arn
            }
        },
        storageConfiguration=storage_configuration
    )

    # Pretty print the response
    pprint.pprint(kb_obj)

except Exception as e:
    print(f"Error occurred: {e}")

{'ResponseMetadata': {'HTTPHeaders': {'connection': 'keep-alive',
                                      'content-length': '974',
                                      'content-type': 'application/json',
                                      'date': 'Mon, 04 Nov 2024 16:16:07 GMT',
                                      'x-amz-apigw-id': 'AurzOGeDPHcEvjg=',
                                      'x-amzn-requestid': '4ebe5cb9-6c40-49a0-92e7-2ec690de8156',
                                      'x-amzn-trace-id': 'Root=1-6728f347-67ee8f1d4e96d0aa51510cad'},
                      'HTTPStatusCode': 202,
                      'RequestId': '4ebe5cb9-6c40-49a0-92e7-2ec690de8156',
                      'RetryAttempts': 0},
 'knowledgeBase': {'createdAt': datetime.datetime(2024, 11, 4, 16, 16, 7, 490731, tzinfo=tzutc()),
                   'description': 'KB that contains the bedrock documentation',
                   'knowledgeBaseArn': 'arn:aws:bedrock:us-west-2:939490215743:knowledge-base/0CEEAN

##TECHNICAL_ANALYSIS

In [37]:



class StockAnalysis:
    def __init__(self, ticker: str, start_date: str, end_date: str):
        """
        Initialize StockAnalysis with a ticker symbol, start date, and end date.

        Parameters:
        ticker (str): The stock ticker symbol (e.g., 'AAPL' for Apple).
        start_date (str): The starting date for historical data in 'YYYY-MM-DD' format.
        end_date (str): The ending date for historical data in 'YYYY-MM-DD' format.
        """
        self.ticker = ticker
        self.start_date = start_date
        self.end_date = end_date
        self.data = None

    def download_data(self) -> None:
        """
        Download historical stock data using the specified ticker symbol and date range.

        This method downloads data with a buffer period for calculating indicators.
        """
        buffer_days = 26
        start_with_buffer = pd.to_datetime(self.start_date) - pd.Timedelta(days=buffer_days)
        self.data = yf.download(self.ticker, start=start_with_buffer.strftime('%Y-%m-%d'), end=self.end_date)
        self.data = self.data.copy()

    def calculate_rsi(self, period: int = 14) -> None:
        """
        Calculate the Relative Strength Index (RSI) for the stock data.

        Parameters:
        period (int): The number of periods to use for the RSI calculation (default is 14).

        Raises:
        ValueError: If data has not been downloaded.
        """
        if self.data is None:
            raise ValueError("Data not downloaded. Please call download_data() first.")
        
        delta = self.data['Close'].diff()
        gains = delta.where(delta > 0, 0)
        losses = -delta.where(delta < 0, 0)
        
        avg_gain = gains.ewm(span=period, min_periods=period).mean()
        avg_loss = losses.ewm(span=period, min_periods=period).mean()
        
        rs = avg_gain / avg_loss
        self.data.loc[:, 'RSI'] = 100 - (100 / (1 + rs))

    def calculate_macd(self, short_window: int = 12, long_window: int = 26, signal_window: int = 9) -> None:
        """
        Calculate the Moving Average Convergence Divergence (MACD) and the Signal Line.

        Parameters:
        short_window (int): The short-term EMA window (default is 12).
        long_window (int): The long-term EMA window (default is 26).
        signal_window (int): The signal line EMA window (default is 9).

        Raises:
        ValueError: If data has not been downloaded.
        """
        if self.data is None:
            raise ValueError("Data not downloaded. Please call download_data() first.")
        
        self.data.loc[:, 'Short_EMA'] = self.data['Close'].ewm(span=short_window, adjust=False).mean()
        self.data.loc[:, 'Long_EMA'] = self.data['Close'].ewm(span=long_window, adjust=False).mean()
        
        self.data.loc[:, 'MACD'] = self.data['Short_EMA'] - self.data['Long_EMA']
        self.data.loc[:, 'Signal_Line'] = self.data['MACD'].ewm(span=signal_window, adjust=False).mean()

        self.data = self.data.loc[self.start_date:].copy()

    def calculate_obv(self) -> None:
        """
        Calculate the On-Balance Volume (OBV) indicator.

        Raises:
        ValueError: If data has not been downloaded.
        """
        if self.data is None:
            raise ValueError("Data not downloaded. Please call download_data() first.")
        
        price_change = self.data['Close'].diff()
        obv = (price_change > 0).astype(int) * self.data['Volume'] - \
              (price_change < 0).astype(int) * self.data['Volume']
        
        self.data.loc[:, 'OBV'] = obv.cumsum()

    def get_rsi_data(self) -> pd.DataFrame:
        """
        Return a DataFrame of dates and their corresponding RSI values.

        Returns:
        pd.DataFrame: A DataFrame containing the Date and RSI values.

        Raises:
        ValueError: If RSI has not been calculated.
        """
        if 'RSI' not in self.data.columns:
            raise ValueError("RSI not calculated. Please call calculate_rsi() first.")
        
        rsi_data = self.data[['RSI']].dropna().reset_index()
        rsi_data.columns = ['Date', 'RSI']
        return rsi_data

    def get_macd_data(self) -> pd.DataFrame:
        """
        Return a DataFrame of dates and their corresponding MACD and Signal Line values.

        Returns:
        pd.DataFrame: A DataFrame containing the Date, MACD, and Signal Line values.

        Raises:
        ValueError: If MACD has not been calculated.
        """
        if 'MACD' not in self.data.columns or 'Signal_Line' not in self.data.columns:
            raise ValueError("MACD not calculated. Please call calculate_macd() first.")
        
        macd_data = self.data[['MACD', 'Signal_Line']].dropna().reset_index()
        macd_data.columns = ['Date', 'MACD', 'Signal Line']
        return macd_data

    def get_obv_data(self) -> pd.DataFrame:
        """
        Return a DataFrame of dates and their corresponding OBV values.

        Returns:
        pd.DataFrame: A DataFrame containing the Date and OBV values.

        Raises:
        ValueError: If OBV has not been calculated.
        """
        if 'OBV' not in self.data.columns:
            raise ValueError("OBV not calculated. Please call calculate_obv() first.")
        
        obv_data = self.data[['OBV']].dropna().reset_index()
        obv_data.columns = ['Date', 'OBV']
        return obv_data

    def plot_rsi(self) -> str:
        """
        Plot RSI and stock price, returning a base64-encoded image string.

        Returns:
        str: A base64-encoded string representing the plot image.

        Raises:
        ValueError: If RSI has not been calculated.
        """
        if self.data is None or 'RSI' not in self.data.columns:
            raise ValueError("RSI not calculated. Please calculate RSI before plotting.")

        fig, ax1 = plt.subplots(figsize=(12, 6))

        ax1.plot(self.data.index, self.data['RSI'], label='RSI', color='blue')
        ax1.axhline(30, color='red', linestyle='dotted', linewidth=1, label='Oversold (30)')
        ax1.axhline(70, color='green', linestyle='dotted', linewidth=1, label='Overbought (70)')
        ax1.set_ylim(0, 100)
        ax1.set_ylabel("RSI")
        ax1.set_title(f"RSI and Stock Price for {self.ticker} from {self.start_date} to {self.end_date}")
        ax1.legend(loc="upper left")
        ax1.grid()

        ax2 = ax1.twinx()
        ax2.plot(self.data.index, self.data['Close'], label='Stock Price', color='black', alpha=0.5)
        ax2.set_ylabel("Stock Price")
        ax2.legend(loc="upper right")

        # Save the plot to a BytesIO object and encode it
        buf = BytesIO()
        plt.savefig(buf, format='png')
        plt.close(fig)
        buf.seek(0)
        return base64.b64encode(buf.getvalue()).decode('utf-8')  # Return base64 string

    def plot_macd(self) -> str:
        """
        Plot MACD and stock price, returning a base64-encoded image string.

        Returns:
        str: A base64-encoded string representing the plot image.

        Raises:
        ValueError: If MACD has not been calculated.
        """
        if self.data is None or 'MACD' not in self.data.columns or 'Signal_Line' not in self.data.columns:
            raise ValueError("MACD not calculated. Please calculate MACD before plotting.")

        fig, ax1 = plt.subplots(figsize=(12, 6))

        ax1.plot(self.data.index, self.data['MACD'], label='MACD', color='blue')
        ax1.plot(self.data.index, self.data['Signal_Line'], label='Signal Line', color='orange')
        ax1.axhline(0, color='black', linestyle='dashed', linewidth=1)
        ax1.set_title(f"MACD and Stock Price for {self.ticker} from {self.start_date} to {self.end_date}")
        ax1.set_ylabel("MACD")
        ax1.legend(loc="upper left")
        ax1.grid()

        ax2 = ax1.twinx()
        ax2.plot(self.data.index, self.data['Close'], label='Stock Price', color='black', alpha=0.5)
        ax2.set_ylabel("Stock Price")
        ax2.legend(loc="upper right")

        # Save the plot to a BytesIO object and encode it
        buf = BytesIO()
        plt.savefig(buf, format='png')
        plt.close(fig)
        buf.seek(0)
        return base64.b64encode(buf.getvalue())


    def plot_obv(self) -> str:
        """
        Plots OBV and stock price, returning a base64-encoded image string.
        """
        if self.data is None or 'OBV' not in self.data.columns:
            raise ValueError("OBV not calculated. Please calculate OBV before plotting.")

        # Create a figure and axis for plotting
        fig, ax1 = plt.subplots(figsize=(12, 6))

        # Plot OBV
        ax1.plot(self.data.index, self.data['OBV'], label='OBV', color='purple')
        ax1.set_ylabel("OBV")
        ax1.set_title(f"OBV and Stock Price for {self.ticker} from {self.start_date} to {self.end_date}")
        ax1.legend(loc="upper left")
        ax1.grid()

        # Create a second y-axis for the stock price
        ax2 = ax1.twinx()
        ax2.plot(self.data.index, self.data['Close'], label='Stock Price', color='black', alpha=0.5)
        ax2.set_ylabel("Stock Price")
        ax2.legend(loc="upper right")

        # Save the plot to a BytesIO object and encode it
        buf = BytesIO()
        plt.savefig(buf, format='png')
        plt.close(fig)  # Close the figure to free up memory
        buf.seek(0)
        return base64.b64encode(buf.getvalue()).decode('utf-8')  # Return base64 string

    def get_fear_and_greed_index(self):
        """
        Retrieves the current Fear and Greed index value.
        """
        return fear_and_greed.get().value

    def get_all_data_summary(self) -> dict:
        """
        Returns a summary of all calculated indicators and the Fear and Greed index
        in a structured dictionary format for easy recognition.
        """
        summary = {
            "Ticker": self.ticker,
            "Start Date": self.start_date,
            "End Date": self.end_date,
            "Fear and Greed Index": self.get_fear_and_greed_index(),
            "RSI Values": [],
            "MACD Values": [],
            "OBV Values": []
        }

        # RSI Data
        if 'RSI' in self.data.columns:
            rsi_data = self.get_rsi_data()  # Ensure this method exists
            for index, row in rsi_data.iterrows():
                summary["RSI Values"].append({"Date": row['Date'].date().isoformat(), "RSI": round(row['RSI'], 2)})
        else:
            summary["RSI Values"].append({"Error": "RSI has not been calculated."})

        # MACD Data
        if 'MACD' in self.data.columns and 'Signal_Line' in self.data.columns:
            macd_data = self.get_macd_data()  # Ensure this method exists
            for index, row in macd_data.iterrows():
                summary["MACD Values"].append({
                    "Date": row['Date'].date().isoformat(),
                    "MACD": round(row['MACD'], 2),
                    "Signal Line": round(row['Signal Line'], 2)
                })
        else:
            summary["MACD Values"].append({"Error": "MACD has not been calculated."})

        # OBV Data
        if 'OBV' in self.data.columns:
            obv_data = self.get_obv_data()  # Ensure this method exists
            for index, row in obv_data.iterrows():
                summary["OBV Values"].append({"Date": row['Date'].date().isoformat(), "OBV": round(row['OBV'], 2)})
        else:
            summary["OBV Values"].append({"Error": "OBV has not been calculated."})

        return summary

    def get_complete_analysis(self) -> dict:
        """
        Downloads data, calculates all indicators (RSI, MACD, OBV), and returns a complete summary.

        Returns:
            dict: A dictionary containing all calculated indicators and the Fear and Greed index.

        Raises:
            ValueError: If there's an error downloading data or calculating indicators.
        """
        try:
            # Download the data
            self.download_data()
    
            # Calculate all indicators
            self.calculate_rsi()  # Ensure this method exists
            self.calculate_macd()  # Ensure this method exists
            self.calculate_obv()
    
            # Get and return the summary
            return self.get_all_data_summary()
    
        except Exception as e:
            raise ValueError(f"Error performing complete analysis: {str(e)}")

    def get_financial_reports(self):
        """
        Retrieves financial reports from Yahoo Finance and formats the output.
        """
        ticker = yf.Ticker(self.ticker)
        reports = {}
        try:
            # Retrieve the financial statements
            income_statement = ticker.income_stmt
            balance_sheet = ticker.balancesheet
            cash_flow_statement = ticker.cashflow

            # Convert DataFrames to lists of dictionaries
            reports['Income Statement'] = income_statement.reset_index().to_dict(orient='records')
            reports['Balance Sheet'] = balance_sheet.reset_index().to_dict(orient='records')
            reports['Cash Flow Statement'] = cash_flow_statement.reset_index().to_dict(orient='records')

            # Convert all Timestamp keys to string
            for report in reports.values():
                for entry in report:
                    for key in list(entry.keys()):
                        if isinstance(key, pd.Timestamp):
                            entry[str(key)] = entry.pop(key)  # Convert Timestamp to string key

        except Exception as e:
            print(f"Error retrieving financial reports: {e}")
            return {}

        return reports




In [38]:
#s3_client.upload_file(local_file_path, bucket_name, os.path.join(kb_key, f))


In [83]:
ticker = 'AAPL'
technical_data = StockAnalysis('VST',"2023-02-13","2024-03-13")

In [ ]:
# Create IAM Role for the agent and attach IAM policies
assume_role_policy_document = {
    "Version": "2012-10-17",
    "Statement": [{
          "Effect": "Allow",
          "Principal": {
            "Service": "bedrock.amazonaws.com"
          },
          "Action": "sts:AssumeRole"
    }]
}

assume_role_policy_document_json = json.dumps(assume_role_policy_document)
agent_role = iam_client.create_role(
    RoleName=agent_role_name,
    AssumeRolePolicyDocument=assume_role_policy_document_json
)

# Pause to make sure role is created
time.sleep(20)
    
iam_client.attach_role_policy(
    RoleName=agent_role_name,
    PolicyArn=agent_bedrock_policy['Policy']['Arn']
)


iam_client.attach_role_policy(
    RoleName=agent_role_name,
    PolicyArn=agent_kb_schema_policy['Policy']['Arn']
)

{'ResponseMetadata': {'RequestId': '00a2c1c8-f9f6-4fb1-89a0-3750fe622c88',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Mon, 04 Nov 2024 16:17:06 GMT',
   'x-amzn-requestid': '00a2c1c8-f9f6-4fb1-89a0-3750fe622c88',
   'content-type': 'text/xml',
   'content-length': '212'},
  'RetryAttempts': 0}}

In [54]:
# Create Agent
response = bedrock_agent_client.create_agent(
    agentName=agent_name,
    agentResourceRoleArn=agent_role['Role']['Arn'],
    description="Agent supporting Amazon Bedrock Developers.",
    idleSessionTTLInSeconds=1800,
    foundationModel=model_id,
    instruction=agent_instruction,
)

In [55]:
agent_id = response['agent']['agentId']
agent_id

'KTTEPR1I17'

In [57]:
agent_kb_description = bedrock_agent_client.associate_agent_knowledge_base(
    agentId=agent_id,
    agentVersion='DRAFT',
    description=f'Use the information in the {kb_name} knowledge base to provide accurate responses to the questions about Amazon Bedrock.',
    knowledgeBaseId=knowledge_base_id 
)

In [58]:
agent_prepare = bedrock_agent_client.prepare_agent(agentId=agent_id)
agent_prepare

{'ResponseMetadata': {'RequestId': 'c1ce04fc-c04c-48b5-9b91-28ee4feabe38',
  'HTTPStatusCode': 202,
  'HTTPHeaders': {'date': 'Mon, 04 Nov 2024 16:17:27 GMT',
   'content-type': 'application/json',
   'content-length': '119',
   'connection': 'keep-alive',
   'x-amzn-requestid': 'c1ce04fc-c04c-48b5-9b91-28ee4feabe38',
   'x-amz-apigw-id': 'Aur_xHfovHcEqKA=',
   'x-amzn-trace-id': 'Root=1-6728f397-5c4476066803f7ac676e4991'},
  'RetryAttempts': 0},
 'agentId': 'KTTEPR1I17',
 'agentStatus': 'PREPARING',
 'agentVersion': 'DRAFT',
 'preparedAt': datetime.datetime(2024, 11, 4, 16, 17, 27, 853294, tzinfo=tzutc())}

In [59]:
# Pause to make sure agent is prepared
time.sleep(30)
agent_alias = bedrock_agent_client.create_agent_alias(
    agentId=agent_id,
    agentAliasName=agent_alias_name
)
# Pause to make sure agent alias is ready
time.sleep(30)

In [60]:
agent_alias


{'ResponseMetadata': {'RequestId': 'e2f6ef9d-7f1c-46dd-8df6-9c7a2d5c2650',
  'HTTPStatusCode': 202,
  'HTTPHeaders': {'date': 'Mon, 04 Nov 2024 16:18:01 GMT',
   'content-type': 'application/json',
   'content-length': '354',
   'connection': 'keep-alive',
   'x-amzn-requestid': 'e2f6ef9d-7f1c-46dd-8df6-9c7a2d5c2650',
   'x-amz-apigw-id': 'AusE9FD9vHcEusA=',
   'x-amzn-trace-id': 'Root=1-6728f3b8-013b465a63117c0319a0daf4'},
  'RetryAttempts': 0},
 'agentAlias': {'agentAliasArn': 'arn:aws:bedrock:us-west-2:939490215743:agent-alias/KTTEPR1I17/R2VDP7TZGT',
  'agentAliasId': 'R2VDP7TZGT',
  'agentAliasName': 'bedrock-docs-alias1730736817',
  'agentAliasStatus': 'CREATING',
  'agentId': 'KTTEPR1I17',
  'createdAt': datetime.datetime(2024, 11, 4, 16, 18, 1, 15735, tzinfo=tzutc()),
  'routingConfiguration': [{}],
  'updatedAt': datetime.datetime(2024, 11, 4, 16, 18, 1, 15735, tzinfo=tzutc())}}

In [61]:
# Extract the agentAliasId from the response
agent_alias_id = agent_alias['agentAlias']['agentAliasId']

## create a random id for session initiator id
session_id:str = str(uuid.uuid1())
enable_trace:bool = True
end_session:bool = False



In [62]:
# invoke the agent API
agentResponse = bedrock_agent_runtime_client.invoke_agent(
    inputText=f"tu vas rediger un texte financier sur{ticker} ",
    agentId=agent_id,
    agentAliasId=agent_alias_id, 
    sessionId=session_id,
    enableTrace=enable_trace, 
    endSession= end_session
)

logger.info(pprint.pprint(agentResponse))

[2024-11-04 11:18:36,801] p197733 {3534585163.py:11} INFO - None


{'ResponseMetadata': {'HTTPHeaders': {'connection': 'keep-alive',
                                      'content-type': 'application/vnd.amazon.eventstream',
                                      'date': 'Mon, 04 Nov 2024 16:18:36 GMT',
                                      'transfer-encoding': 'chunked',
                                      'x-amz-bedrock-agent-session-id': '7092a693-9ac8-11ef-858c-90e868d4499b',
                                      'x-amzn-bedrock-agent-content-type': 'application/json',
                                      'x-amzn-requestid': 'e5034c58-81c4-4f69-a816-6345b37ae31e'},
                      'HTTPStatusCode': 200,
                      'RequestId': 'e5034c58-81c4-4f69-a816-6345b37ae31e',
                      'RetryAttempts': 0},
 'completion': <botocore.eventstream.EventStream object at 0x71d751bb0da0>,
 'contentType': 'application/json',
 'sessionId': '7092a693-9ac8-11ef-858c-90e868d4499b'}


In [63]:
%%time
event_stream = agentResponse['completion']
try:
    for event in event_stream:        
        if 'chunk' in event:
            data = event['chunk']['bytes']
            logger.info(f"Final answer ->\n{data.decode('utf8')}")
            agent_answer = data.decode('utf8')
            end_event_received = True
            # End event indicates that the request finished successfully
        elif 'trace' in event:
            logger.info(json.dumps(event['trace'], indent=2))
        else:
            raise Exception("unexpected event.", event)
except Exception as e:
    raise Exception("unexpected event.", e)

[2024-11-04 11:18:39,077] p197733 {<timed exec>:11} INFO - {
  "agentAliasId": "R2VDP7TZGT",
  "agentId": "KTTEPR1I17",
  "agentVersion": "1",
  "sessionId": "7092a693-9ac8-11ef-858c-90e868d4499b",
  "trace": {
    "orchestrationTrace": {
      "invocationInput": {
        "invocationType": "KNOWLEDGE_BASE",
        "knowledgeBaseLookupInput": {
          "knowledgeBaseId": "0CEEANELN8",
          "text": "tu vas rediger un texte financier surTSLA "
        },
        "traceId": "e5034c58-81c4-4f69-a816-6345b37ae31e-0"
      }
    }
  }
}
[2024-11-04 11:18:39,079] p197733 {<timed exec>:11} INFO - {
  "agentAliasId": "R2VDP7TZGT",
  "agentId": "KTTEPR1I17",
  "agentVersion": "1",
  "sessionId": "7092a693-9ac8-11ef-858c-90e868d4499b",
  "trace": {
    "orchestrationTrace": {
      "observation": {
        "knowledgeBaseLookupOutput": {
          "retrievedReferences": []
        },
        "traceId": "e5034c58-81c4-4f69-a816-6345b37ae31e-0",
        "type": "KNOWLEDGE_BASE"
      }
    }

CPU times: user 7.47 ms, sys: 132 μs, total: 7.6 ms
Wall time: 5.54 s


In [64]:
def simple_agent_invoke(input_text, agent_id, agent_alias_id, session_id=None, enable_trace=False, end_session=False):
    if session_id is None:
        session_id:str = str(uuid.uuid1())

    agentResponse = bedrock_agent_runtime_client.invoke_agent(
        inputText=input_text,
        agentId=agent_id,
        agentAliasId=agent_alias_id, 
        sessionId=session_id,
        enableTrace=enable_trace, 
        endSession= end_session
    )
    logger.info(pprint.pprint(agentResponse))
    
    agent_answer = ''
    event_stream = agentResponse['completion']
    try:
        for event in event_stream:        
            if 'chunk' in event:
                data = event['chunk']['bytes']
                logger.info(f"Final answer ->\n{data.decode('utf8')}")
                agent_answer = data.decode('utf8')
                end_event_received = True
                # End event indicates that the request finished successfully
            elif 'trace' in event:
                logger.info(json.dumps(event['trace'], indent=2))
            else:
                raise Exception("unexpected event.", event)
    except Exception as e:
        raise Exception("unexpected event.", e)
    return agent_answer

In [65]:
reponse = simple_agent_invoke(f"que peux tu me dire sur {ticker}", agent_id, agent_alias_id, session_id)


[2024-11-04 11:18:52,752] p197733 {321621934.py:13} INFO - None


{'ResponseMetadata': {'HTTPHeaders': {'connection': 'keep-alive',
                                      'content-type': 'application/vnd.amazon.eventstream',
                                      'date': 'Mon, 04 Nov 2024 16:18:52 GMT',
                                      'transfer-encoding': 'chunked',
                                      'x-amz-bedrock-agent-session-id': '7092a693-9ac8-11ef-858c-90e868d4499b',
                                      'x-amzn-bedrock-agent-content-type': 'application/json',
                                      'x-amzn-requestid': '21707daa-f8cd-406a-962e-1d7ad6052efb'},
                      'HTTPStatusCode': 200,
                      'RequestId': '21707daa-f8cd-406a-962e-1d7ad6052efb',
                      'RetryAttempts': 0},
 'completion': <botocore.eventstream.EventStream object at 0x71d7516c4890>,
 'contentType': 'application/json',
 'sessionId': '7092a693-9ac8-11ef-858c-90e868d4499b'}


[2024-11-04 11:18:59,875] p197733 {321621934.py:21} INFO - Final answer ->
Tesla est une entreprise américaine qui conçoit et fabrique des véhicules électriques haut de gamme, des composants pour véhicules électriques et des produits d'énergie solaire et de stockage d'énergie. Ses principaux produits sont les voitures électriques Model S, Model X, Model 3 et Model Y, ainsi que les systèmes de stockage d'énergie Powerwall et Powerpack. Sur le plan financier, Tesla a connu une croissance rapide ces dernières années mais reste déficitaire. En 2022, Tesla a réalisé un chiffre d'affaires de 81,5 milliards de dollars mais une perte nette de 2,1 milliards de dollars. Les principaux défis de Tesla sont la concurrence croissante sur le marché des véhicules électriques, les problèmes de chaîne d'approvisionnement et les coûts élevés de production.


In [66]:
rp1 = reponse

In [67]:
modelId = 'anthropic.claude-3-sonnet-20240229-v1:0'
bedrock_client = boto3.client('bedrock-agent-runtime', region_name='us-west-2')
bedrock_client = boto3.client(service_name='bedrock-runtime', region_name='us-west-2')


In [ ]:

# Import python's built-in regular expression library
# Import the hints module from the utils package


In [68]:
def get_completion(prompt, system_prompt=None):
    # Define the inference configuration
    inference_config = {
        "temperature": 0.0,  # Set the temperature for generating diverse responses
        "maxTokens": 500,  # Set the maximum number of tokens to generate
        "topP": 1,  # Set the top_p value for nucleus sampling
    }
    # Create the converse method parameters
    converse_api_params = {
        "modelId": modelId,  # Specify the model ID to use
        "messages": [{"role": "user", "content": [{"text": prompt}]}],  # Provide the user's prompt
        "inferenceConfig": inference_config,  # Pass the inference configuration
    }
    # Check if system_text is provided
    if system_prompt:
        # If system_text is provided, add the system parameter to the converse_params dictionary
        converse_api_params["system"] = [{"text":system_prompt}]
            # Send a request to the Bedrock client to generate a response
    try:
        response = bedrock_client.converse(**converse_api_params)

        # Extract the generated text content from the response
        text_content = response['output']['message']['content'][0]['text']

        # Return the generated text content
        return text_content

    except ClientError as err:
        message = err.response['Error']['Message']
        print(f"A client error occured: {message}")



In [69]:
import boto3

def list_objects_in_bucket(bucket_name):
    """Liste les fichiers dans le bucket S3 spécifié."""
    s3 = boto3.client('s3')
    
    try:
        # Récupère la liste des objets dans le bucket
        response = s3.list_objects_v2(Bucket=bucket_name)
        
        if 'Contents' in response:
            print("Fichiers dans le bucket :")
            for obj in response['Contents']:
                print(obj['Key'])  # Affiche le nom de l'objet (object key)
        else:
            print("Aucun objet trouvé dans le bucket.")
    
    except Exception as e:
        print(f"Une erreur s'est produite : {e}")

# Exemple d'utilisation
if __name__ == "__main__":
    bucket_name = "your-bucket-name"  # Remplacez par le nom de votre bucket
    list_objects_in_bucket(bucket_name)


Une erreur s'est produite : An error occurred (AllAccessDisabled) when calling the ListObjectsV2 operation: All access to this object has been disabled


In [ ]:
from flask import Flask, request, jsonify
from flask_cors import CORS
import yfinance

app = Flask(__name__)
CORS(app)

@app.route('/api/ticker', methods=['POST'])
def get_ticker():
    data = request.get_json()
    ticker = data['ticker']
    tech_data = StockAnalysis(ticker,"2023-02-13","2024-03-13")
    FINANCIAL_REPORTS = tech_data.get_financial_reports()
    TECHNICAL_DATA= tech_data.get_complete_analysis()
    prompt_systeme_fundamental_analysis = """<task_context>
    You are a financial analyst specializing in stock market evaluations. Your goal is to conduct a comprehensive fundamental analysis of a given stock based on the financial statements data retrieved as a single variable. This analysis should be purely fundamental, separate from any technical analysis or general information provided from previous evaluations.
    </task_context>

    <tone_context>
    Maintain a professional and analytical tone throughout your response.
    </tone_context>

    <background_data>
    The following financial data will be provided as a single dictionary:
    <financial_reports>"""+str(FINANCIAL_REPORTS)+"""</financial_reports>
    </background_data>

    <detailed_task_description_and_rules>
    - Analyze the provided financial statements to assess the stock's performance, profitability, liquidity, and overall financial health.
    - Focus on key financial ratios (e.g., P/E ratio, debt-to-equity ratio, current ratio) and current trends in the financial data.
    - Ensure your analysis is clear and concise, highlighting significant insights and conclusions without discussing historical value movements.
    - Provide a recommendation for action (buy, hold, or sell) based solely on your fundamental analysis.
    - If you are unsure about any aspect of the analysis, say "I'm not sure" and avoid making assumptions.
    </detailed_task_description_and_rules>

    <immediate_task_description_or_request>
    Given the financial data provided, perform a fundamental analysis of the stock and structure the output in JSON format.
    </immediate_task_description_or_request>

    <think_step_by_step>
    Please think through your analysis step-by-step before delivering your final response.
    </think_step_by_step>

    <output_formatting>
    The output should be structured as a JSON object with one key named "key1" containening relevant sections for the fundamental analysis , give directly  the result of it , dont tell what you are gonna do give the answer directly font desribe each title , tell me just information related to the company 
    , including:
    - "income_statement_analysis"
    - "quarterly_income_statement_analysis"
    - "balance_sheet_analysis"
    - "quarterly_balance_sheet_analysis"
    - "cash_flow_statement_analysis"
    - "quarterly_cashflow_analysis"
    - "action_recommendation"
    </output_formatting>

    <prefilled_response>
    <response>
    {
        "key1": {
            "income_statement_analysis": {
                "section_title": "Income Statement Analysis",
                "section_content": "Provide a detailed analysis of the income statement here."
            },
            "quarterly_income_statement_analysis": {
                "section_title": "Quarterly Income Statement Analysis",
                "section_content": "Provide a detailed analysis of the quarterly income statement here."
            },
            "balance_sheet_analysis": {
                "section_title": "Balance Sheet Analysis",
                "section_content": "Provide a detailed analysis of the balance sheet here."
            },
            "quarterly_balance_sheet_analysis": {
                "section_title": "Quarterly Balance Sheet Analysis",
                "section_content": "Provide a detailed analysis of the quarterly balance sheet here."
            },
            "cash_flow_statement_analysis": {
                "section_title": "Cash Flow Statement Analysis",
                "section_content": "Provide a detailed analysis of the cash flow statement here."
            },
            "quarterly_cashflow_analysis": {
                "section_title": "Quarterly Cash Flow Analysis",
                "section_content": "Provide a detailed analysis of the quarterly cash flow here."
            },
            "action_recommendation": {
                "section_title": "Action Recommendation",
                "section_content": "Based on the analysis, recommend whether to buy, hold, or sell the stock."
            }
        }
    }
    </response>
    </prefilled_response>"""
    pre_prompt_technical_analysis ="""
    <task_context>
    You are a financial analyst specializing in stock market evaluations. Your goal is to conduct a comprehensive technical analysis of a given stock based on the technical indicators calculated from the historical stock data. This analysis should be purely technical and must not include any fundamental analysis or general insights from previous evaluations.
    </task_context>

    <tone_context>
    Maintain a professional and analytical tone throughout your response.
    </tone_context>

    <background_data>
    The following technical data will be provided as a single dictionary:
    <technical_data>"""+str(TECHNICAL_DATA)+"""</technical_data>
    </background_data>

    <detailed_task_description_and_rules>
    - Analyze the provided technical indicators, including the Fear and Greed Index, to assess the stock's price momentum, trend direction, and potential buy/sell signals.
    - Focus on key indicators such as Relative Strength Index (RSI), Moving Average Convergence Divergence (MACD), On-Balance Volume (OBV), and the Fear and Greed Index.
    - Ensure your analysis is concise and highlights significant insights, such as current levels, crossovers, divergences, and trends without discussing historical patterns or oscillations.
    - Provide a clear recommendation for action (buy, hold, or sell) based on current market conditions.
    - If you are unsure about any aspect of the analysis, say "I'm not sure" and avoid making assumptions.
    </detailed_task_description_and_rules>

    <immediate_task_description_or_request>
    Given the technical data provided, perform a technical analysis of the stock and structure the output in JSON format, including a recommendation for action based on your findings , give the answer directly font desribe each indicator , tell me just the meaning of its value 
    </immediate_task_description_or_request>

    <think_step_by_step>
    Please think through your analysis step-by-step before delivering your final response.
    </think_step_by_step>

    <output_formatting>
    The output should be structured as a JSON object with one specific  key named key2 containing relevant sections for the technical analysis, including:
    - "rsi_analysis"
    - "macd_analysis"
    - "obv_analysis"
    - "fear_and_greed_index_analysis"
    - "general_trends"
    - "action_recommendation"
    </output_formatting>

    <prefilled_response>
    <response>
    { "key2": {
        "rsi_analysis": "Current RSI level: [RSI value]. [Insight on RSI level].",
        "macd_analysis": "Current MACD level: [MACD value]. [Insight on MACD level].",
        "obv_analysis": "Current OBV level: [OBV value]. [Insight on OBV level].",
        "fear_and_greed_index_analysis": "Current Fear and Greed Index: [Index value]. [Insight on index value].",
        "general_trends": "General trends: [Insight on current price momentum].",
        "action_recommendation": "[Recommendation for action based on current market conditions]."
    }

    </response>
    </prefilled_response>"""
    SYSTEM_PROMPT_1 = prompt_systeme_fundamental_analysis
    SYSTEM_PROMPT_2 = pre_prompt_technical_analysis
    #SYSTEM_PROMPT_1 = 
    #pre_prompt_technical_analysis ,prompt_systeme_fundamental_analysis

    # Prompt
    PROMPT = ticker

    # Print Claude's response
    rep_1 = get_completion(PROMPT , SYSTEM_PROMPT_1)
    rep_2 = get_completion(PROMPT , SYSTEM_PROMPT_2)
    print(rep_1)
    return jsonify({'fund': str(rep_1), 'tech': str(rep_2)}), 200

@app.route('/yfinance/get_price', methods=['POST'])
def get_price():
    data = request.get_json()
    symbol = data.get("ticker")
    ticker = yfinance.Ticker(symbol)
    if ticker.info['trailingPegRatio'] == None:
        return jsonify({'error':'error'}), 400
    return jsonify({'price':f"{ticker.info['currentPrice']:.2f}"}), 200

@app.route('/yfinance/get_stocks', methods=['GET'])
def get_stocks():
    tickers = yfinance.Tickers("AAPL TSLA MSFT NVDA")
    infos = []
    for symbol, ticker in tickers.tickers.items():
        data = ticker.info
        infos.append({
            "symbol":symbol,
            "name":data['shortName'],
            'price': f"{data['currentPrice']:.2f}",
            'high': data['dayHigh'],
            'low':data['dayLow'],
            'target':data['targetMeanPrice'],
            'recommendation':data['recommendationKey']
        })
    return jsonify({'infos':infos}), 200

if __name__ == '__main__':
    app.run(port=5000)




 * Serving Flask app '__main__'
 * Debug mode: off


[2024-11-04 12:03:06,482] p197733 {_internal.py:97} INFO - WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
[2024-11-04 12:03:06,483] p197733 {_internal.py:97} INFO - Press CTRL+C to quit
[2024-11-04 12:03:14,258] p197733 {_internal.py:97} INFO - 127.0.0.1 - - [04/Nov/2024 12:03:14] "GET /yfinance/get_stocks HTTP/1.1" 200 -
[2024-11-04 12:03:17,155] p197733 {_internal.py:97} INFO - 127.0.0.1 - - [04/Nov/2024 12:03:17] "OPTIONS /yfinance/get_price HTTP/1.1" 200 -
[2024-11-04 12:03:17,165] p197733 {_internal.py:97} INFO - 127.0.0.1 - - [04/Nov/2024 12:03:17] "POST /yfinance/get_price HTTP/1.1" 200 -
[2024-11-04 12:03:17,185] p197733 {_internal.py:97} INFO - 127.0.0.1 - - [04/Nov/2024 12:03:17] "OPTIONS /api/ticker HTTP/1.1" 200 -
[*********************100%***********************]  1 of 1 completed
[2024-11-04 12:03:49,239] p197733 {_internal.py:97} INFO - 127.0.0.1 - - [04/Nov/2024 1

Here is my fundamental analysis of the stock based on the provided financial data:

{
"key1": {
    "income_statement_analysis": {
        "section_content": "Apple's total revenue has been steadily increasing year-over-year, reaching $391 billion in 2024. The company's gross profit margin is healthy at around 46%. Operating expenses are well managed, with research and development expenses accounting for around 8% of revenue. Net income has grown from $94.7 billion in 2021 to $93.7 billion in 2024, indicating consistent profitability."
    },
    "quarterly_income_statement_analysis": {
        "section_content": "The quarterly income statement data is not provided in the given financial reports."
    },
    "balance_sheet_analysis": {
        "section_content": "Apple has a strong balance sheet with total assets of $365 billion in 2024. The company has a significant cash position of $65.2 billion and investments of $91.5 billion. Total debt stands at $106.6 billion, which is manageabl

In [ ]:
#pre_prompt_general_data = 